In [24]:
import pandas as pd
import numpy as np

In [25]:
# Set random seed for reproducibility
np.random.seed(42)

In [26]:
# Parameters
num_players = 1000
levels = 10
actions = ['play', 'quit', 'restart', 'level_up', 'get_help']
difficulty_levels = ['easy', 'medium', 'hard']

In [27]:
# Generate synthetic player data
data = {
    'player_id': np.arange(1, num_players + 1),
    'level_completed': np.random.randint(1, levels + 1, num_players),
    'time_spent': np.random.randint(1, 301, num_players),  # time in seconds
    'actions_taken': np.random.choice(actions, num_players),
    'difficulty': np.random.randint(1, 7, num_players),  # difficulty now ranges from 1 to 6,
    #'score': np.random.randint(0, 1001, num_players),
    'last_level_attempts': np.random.randint(1, 11, num_players),  # number of attempts (1 to 10)

}

In [28]:
# Create DataFrame
df = pd.DataFrame(data)

# Calculate average time spent per level achieved
df['average_level_time'] = df['time_spent'] / df['level_completed']

# Assign last_level_cleared: 1 if max level is completed, 0 otherwise
df['last_level_cleared'] = np.where(df['level_completed'] == levels, 1, 0)

# Calculate score based on the new formula
df['score'] = (50 * df['level_completed']) + (2 * df['difficulty'])

In [29]:
# Save synthetic dataset to CSV
df.to_csv('synthetic_game_data.csv', index=False)

print("Synthetic dataset created successfully!")

Synthetic dataset created successfully!


In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [31]:
# Load the dataset
df = pd.read_csv('synthetic_game_data.csv')


In [32]:
# Feature engineering for Next Level recommendations
def recommend_nlod(row):
    if row['last_level_attempts'] > 5 and row['last_level_cleared'] == 0:
        new_difficulty = max(1, row['difficulty'] - 1)  # Lower difficulty level
        new_level = row['level_completed']  # Stay on the same level
    elif row['last_level_cleared'] == 1:
        new_difficulty = min(6, row['difficulty'] + 1)  # Next level difficulty
        new_level = row['level_completed'] + 1  # Suggest next level
    else:
        new_difficulty = row['difficulty']  # Same difficulty
        new_level = row['level_completed']  # Stay on the same level

    return new_level, new_difficulty


In [33]:
# Apply the recommendation logic
df[['Next_level', 'NLOD']] = df.apply(recommend_nlod, axis=1, result_type='expand')

In [34]:
0# Prepare features and target variable
X = df[['last_level_attempts', 'last_level_cleared', 'difficulty']]
y = df['NLOD']

In [35]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# Initialize the models
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Support Vector Regression': SVR()
}

In [37]:
# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluation
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Calculate accuracy
    accuracy = model.score(X_test, y_test)  # R² score as a measure of accuracy

    print(f"{name} - Accuracy: {accuracy:.2f}")

Linear Regression - Accuracy: 0.96
Decision Tree - Accuracy: 1.00
Random Forest - Accuracy: 1.00
Support Vector Regression - Accuracy: 0.99


In [38]:
import joblib

# Save each model
joblib.dump(models['Linear Regression'], 'Linear_Regression.pkl')
joblib.dump(models['Decision Tree'], 'Decision_Tree.pkl')
joblib.dump(models['Random Forest'], 'Random_Forest.pkl')
joblib.dump(models['Support Vector Regression'], 'SVR.pkl')

print("Models saved successfully!")

Models saved successfully!


In [39]:
# Save the updated dataset with Next Level recommendations
df.to_csv('synthetic_game_data_with_nlod.csv', index=False)

print("NLOD recommendations added, models evaluated, and final dataset saved successfully!")

NLOD recommendations added, models evaluated, and final dataset saved successfully!
